# Download ERA5 hourly temperatures and generate daily summary

## Method

- Download hourly data for one whole year from ERA5 at 0.5deg resolution as GRIB file (in theory should be faster/smaller than netCDF and we will discard it anyway)
- Calculate minimum daily temperature for each day and save
- Delete the input GRIB to avoid acumulating terabytes of data!

In [1]:
from collections import namedtuple
from pathlib import Path

import numpy as np
import xarray as xr
import cdsapi

In [2]:
DATA_SRC = Path('~/Shared/Data/').expanduser()

In [3]:
SUBDAILY_TEMPERATURES_FOLDER = DATA_SRC / 'weather' / 'era5_0.25deg' / 'hourly_temperature_2m'
SUBDAILY_TEMPERATURES_FOLDER.mkdir(exist_ok=True)

TEMPERATURE_SUMMARY_FOLDER = DATA_SRC / 'weather' / 'era5_0.25deg' / 'daily_temperature_summary'
TEMPERATURE_SUMMARY_FOLDER.mkdir(exist_ok=True)

In [4]:
MAX_YEAR = 2022

In [5]:
assert SUBDAILY_TEMPERATURES_FOLDER.is_dir()
assert TEMPERATURE_SUMMARY_FOLDER.is_dir()

In [7]:
def retreive_year(out_file, year):
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type':'reanalysis',
            'variable':'2m_temperature',
            'year': year,
#             'grid':'0.5/0.5', now get the default rez which is 0.25deg
            'month':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12'
            ],
            'day':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12',
                '13','14','15',
                '16','17','18',
                '19','20','21',
                '22','23','24',
                '25','26','27',
                '28','29','30',
                '31'
            ],
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ],
            'format':'grib'
        },
        str(out_file))
    
    
def generate_daily_summary(source_file):
    # resample and save a single year.
    # Force loading at the start - idea is that once loaded we re-use the data anyway
    # for min/max/mean
    daily = xr.open_dataset(source_file, engine='cfgrib').load()
    daily = daily.resample(time='1D')
    tmin = daily.min()
    tmax = daily.max()
    tmean = daily.mean()

    tmin = tmin.rename({'t2m': 't_min'})
    tmax = tmax.rename({'t2m': 't_max'})
    tmean = tmean.rename({'t2m': 't_mean'})
    daily_summary = xr.merge([tmin, tmax, tmean])
    return daily_summary


def download_and_summarise_year(year, overwrite=False):
    '''Download the GRIB file for hourly temperatures for one year
    Generate a summary file with temperature min, max, mean
    
    TODO: later, probably can delete the hourly data file after summarising it
    '''
    year = str(year)
    out_file = SUBDAILY_TEMPERATURES_FOLDER / f'{year}_temperature.grib'
    summary_file = TEMPERATURE_SUMMARY_FOLDER / f'{year}_temperature_summary.nc'
    
    if overwrite is True or out_file.exists() is False:
        retreive_year(out_file, year)
    else:
        print(f'Skip {out_file}, already exists.')
    
    if overwrite is True or not summary_file.exists():
        daily_summary = generate_daily_summary(out_file)
        
        # Encode using rounding to nearest 0.01K to save space, looses a small amount of precision
        # but should be ok since it not used for detailed simulations...
        daily_summary.to_netcdf(summary_file,
                            encoding={
                                't_min': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999},
                                't_max': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999},
                                't_mean': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999}
                            })
                           
    else:
        print(f'Skip {summary_file}, already exists.')

In [8]:
# EXAMPLE code for triggering many requests.
# joblib.Parallel(n_jobs=10, verbose=10)(
#     joblib.delayed(download_and_summarise_year)(year) for year in range(1980, 2020))

# Run each year one at a time

Because this is anyway a long process, only do one year at a time - just set it going and leave it!

Also, for the summarizing step the whole dataset is loaded into memory because it is anyway needed to get min/max/mean. So can't really do several in parallel b/c it will blow up the RAM!

In [ ]:
for year in range(1980, MAX_YEAR+1):
    print(year)
    download_and_summarise_year(year)

2023-02-01 14:41:43,228 INFO Welcome to the CDS
2023-02-01 14:41:43,230 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-02-01 14:41:43,310 INFO Request is queued


1980
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/hourly_temperature_2m/1980_temperature.grib, already exists.
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/daily_temperature_summary/1980_temperature_summary.nc, already exists.
1981
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/hourly_temperature_2m/1981_temperature.grib, already exists.
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/daily_temperature_summary/1981_temperature_summary.nc, already exists.
1982
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/hourly_temperature_2m/1982_temperature.grib, already exists.
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/daily_temperature_summary/1982_temperature_summary.nc, already exists.
1983
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/hourly_temperature_2m/1983_temperature.grib, already exists.
Skip /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/daily_temperature_summary/1983_temp

2023-02-01 14:41:44,332 INFO Request is running
2023-02-01 14:44:34,698 INFO Request is completed
2023-02-01 14:44:34,701 INFO Downloading https://download-0014-clone.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.internal-1675258904.3244033-19218-4-a42ec8bd-e454-49ac-b974-98a3d0cb716f.grib to /home/jonathanchambers/Shared/Data/weather/era5_0.25deg/hourly_temperature_2m/2022_temperature.grib (16.9G)
2023-02-01 14:58:59,803 INFO Download rate 20.1M/s   
